# Case Study 3
## Description 
As a researcher, I want the ability to design a study/field campaign in advance, so that the best results can be obtained from that study/campaign. 
## Case Breakdown 
- **Actors:** Researcher
- **Goals:** Design a campaign based on maximising variation between stusy sitrs
- **Scope:** Regional, plot-based
- **Language:** R
- **Extra:** TBD
## Generalised case
I want to analyse a suite of spatial variables for a set of candidate locations so I can find a subset that maximises the spread across environments.
## Comparable cases
- I want to combine a suite of environmental variables, including weather (temperature, rainfall, humidity - all xyt), lunar phase (t) and greenness (xyt - see https://portal.tern.org.au/metadata/TERN/8542d90e-6e20-4ad8-b30d-0a171b61d3f5) and soils (pH, N, K, structure) for N (e.g. 50) candidate sites and then select the five sites that maximise spread across different environmental conditions.
- I have a set of study sites in an area and wish to know the best location for adding one more, based on ensuring that the new site adds as much as possible to the range of environments studied.
## Stakeholders 
- **Name:** TBD
- **Contact:** donald.hobern@adelaide.edu.au


## Data Sources - <span style="color:red">DRAFT</span>
- **Reference:** https://appf-central.atlassian.net/l/cp/7yrLwqX1
**UPDATED PLAN 2025-02-07**
The primary goal for this case study is to demonstrate being able to import a set of environmental values for different sites and then use these to identify a subset that maximises spread across the various environmental dimensions.

I think we can keep this simple and include say 5 environmental attributes imported for all Australia (or a subset like NSW) at a moderate grid scale (this needs to be fine-tuned). Suggested datasets:

1. Soil water pH (0-5cm) - see https://esoil.io/TERNLandscapes/Public/Pages/SLGA/ProductDetails-SoilAttributes.html
2. Soil organic carbon (0-5cm) - see https://esoil.io/TERNLandscapes/Public/Pages/SLGA/ProductDetails-SoilAttributes.html
3. Annual mean rainfall
4. Annual maximum temperature
5. Annual minimum temperature

The first two can probably be imported directly as layers from SLGA. The others are probably available online in suitable forms at a slightly better grid resolution and could be imported directly. A nice enhancement to this case study could be to start by importing annual layers (say for the last 5 or 10 years) for each of these and then generating the mean, maximum and minimum respectively for each pixel across the selected years and using that computed output as the import into the main datacube.

Latitude and longitude could be easy additions if we want to experiment with extra variables later.

<span style="color:silver">
**PREVIOUS NOTES**
Data Source 1: SILO Climate DATABASE 
- **Description:** SILO Climate Database for environmental data.
- **Format:** TBD
- **Access Method:** https://www.longpaddock.qld.gov.au/silo/
Data Source 2: Extreme Weather Data (BOM)
- **Description:** Extreme weather data from the Bureau of Meteorology (BOM).
- **Format:** TBD
- **Access Method:** http://www.bom.gov.au/akamai/https-redirect.html
Data Source 1: On-Ground Site
- **Description:** Data from where the chickpeas come from.
- **Format:** TBD
- **Access Method:** TBD
Data Catalog - DRAFT
Describe what kind of data will be using 
</span>

## Data Transformation - <span style="color:red">DRAFT</span> 
**UPDATED PLAN 2025-02-07**
Align/scale all sources on a single grid.

In [4]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## DataCube Generation - <span style="color:red">DRAFT</span> 
Describe the steps take to process the data. 

In [5]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## Data Analysis/Visulisation - <span style="color:red">DRAFT</span> 
**UPDATED PLAN 2025-02-07**
Select 10 random positions across the space selected. 

Find the optimal position for an 11th site to maximise variation across the environment.

Simplest algorithm would be to scale all environmental layers on a 0.0 to 1.0 scale based on the actual range of values in the layer.

Treat these as orthogonal axes in a 5-dimensional space. The environmental distance between any two points will be the fifth root of the product of these five numbers.

We can generate 10 new layers that each represents the environmental distance between each pixel and one of the 10 current points.

We can then find the lowest value for each pixel from these 10 layers - that is the environmental distance between that point and the environmentally closest of the 10 current points.

We can then find which pixel has the highest value in this last layer - that is the most environmentally distant point and the one that should next be surveyed.


In [6]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## NonFunctional Requirements - <span style="color:red">DRAFT</span> 
### Welcome PAGE
#### Objectives
- Showing users what they can do and get and how they can proceed.
- Showing the most updeted information. 
#### Requirements 
- Create a simple UI/Template for user input their requirements
  - showing the normal process of querying data and provide sample data
  - showing the sample output users are expected to have
  - showing the resources users can use
- Creating a 'HELP' Page for users to looking for what kind of attributes they can use
  - Including all the attributes they can query from the data sources
  - Including all the functions users can use and expecting outputs of those query
  - Including informations for asking questions and reaching for help.
- Creating a 'Annoucement' Page for showing the most updated information of the project
  - Including versions/Time/Detail

### User Query 
#### Objectives
- take input from users and process with related data
- Analyze the data and give feedback of what is expected
  - Error Control
  - Time estimation
  - Output Estimatation
#### Requirements 
- Using basic Query language to query DATA
    > Select Phenotype*
    > from On-GroundSite
    > Where CropType = 'Chickpeas'

    > Select Environmental
    > from SILO
    > where weather ='Extreme'- ERROR Control
- Estimation for time & Output
    > ----> Estimated Files intotal: 500 file 5GB
    > ----> Estimated Time:5 mins 50%
    > ----> Noted it may depends on your download speed balabla.
### Error Control
#### Objectives
- State clear what is happening while using try to query
- Give advice & suggestions

### Extras
#### Non-Functional Requirements
- Ensure querying time is within a set time (less than 1 second).
- Create a waiting process for querying if time is not stable.
- Present outputs in a consistent and organized manner.
- Handle unexpected errors gracefully, showing error codes and continuing with other feasible data.